<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [4]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

import pandas as pd
import numpy as np
import random
import sys
import os

In [11]:
df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/DS-Unit-4-Sprint-3-Deep-Learning/master/module1-rnn-and-lstm/sonnets.csv',
                 sep = '\n', header = None)
df.head()

,0
0,1
1,"From fairest creatures we desire increase,"
2,"That thereby beauty’s rose might never die,"
3,"But as the riper should by time decease,"
4,His tender heir might bear his memory:


In [14]:
# Encode Data as Chars
text = " ".join(df[0].tolist())
chars = list(set(text))

char_int = {c:i for i,c in enumerate(chars)}
int_char = {i:c for i,c in enumerate(chars)}

# Resources and Stretch Goals

In [17]:
len(chars)

71

In [18]:
# Create the Sequence Data
# The lists are: encoded (chars encoded as integers)
#                sequences (40 chars per entry)
#                next_chars ()
maxlen = 40
step = 5

encoded = [char_int[c] for c in text]

sequences = [] # Each element is 40 chars long
next_chars = [] # One element for each sequence for prediction

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_chars.append(encoded[i + maxlen])
    
print('sequences', len(sequences))

sequences 18950


In [19]:
# Specify x & y - 3D matrix

# Initialize arrays with False
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_chars[i]] = 1

In [20]:
# build the model: a single LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars)))) #input shape of a singular observ.
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

W0209 11:18:06.673053 4374568384 deprecation.py:506] From /Users/JKMacBook/opt/anaconda3/envs/NN/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [21]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [24]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_int[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = int_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [25]:
model.fit(x, y,
          batch_size=128,
          epochs=20,
          callbacks=[print_callback])

Epoch 1/20
18944/18950 [============================>.] - ETA: 0s - loss: 2.9141
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "your feature.   Incapable of more, reple"
your feature.   Incapable of more, reple th the s t t o te  o e to the tr te the th th the the t the t t re to the t r te t th tie she te tin th ther th the te t the t the t the the  the s th se te the t t  te t te  t e t t  t t t t t the the t the th s the t  te the te t t t t t re te the t the tr th te the  te the te to t to te t the t the thee  te t the the he t the the t t the ti t t te t the te te t s  t t the re t te the to the t 
----- diversity: 0.5
----- Generating with seed: "your feature.   Incapable of more, reple"
your feature.   Incapable of more, repleum ore rs hee  ven th th tind e s n i tre  rieil ito ro  ie ehe t teise ottd sen reos ohn sin st t eoat  minon tolwtr Wo ttorae feh  n te eoiho ile bbecr the  tite tll rs he tet ea me nts r sor tyed s ol he s  hee te 

e, Art left the prey of every vulgar thine or thaues afae Whealk daI tfund coveres medncos sof fat, Htov  rinen hnou I me ring tof gmaicd beaP hanu, wimo  isng anow ut um, mo bile bet baw sre paed, basvse tha’ thyis matyh yoad the hono nhukham lnat bil heprt hertgof eir laol eod foc wich apen, af tiil ched th nn innB ut iud Ang dans dafrta thone , borean. 1nd vet aokurd wange mancPou2 aafWhe this in hice, Tosis ap  ar ro kidr pemeriy hh
----- diversity: 1.2
----- Generating with seed: "e, Art left the prey of every vulgar thi"
e, Art left the prey of every vulgar thire.  moIe  hoaf te aw nd the, Anel wwrad Hte srece Bbem bged pryeuupavlo  ity I thas Fnd moucc weve   ir f bese bode thonw Thetou garsthece tokd’d g eilt masdy Woeway sor mwnlrthec dy1ur thet: Bathor ?fn soe ttuuiR wokfoWh Wunr’e, toutotd fhedrn. Be, go wo h fou d lgrnehe Redd(, al fohn, ghlwhand orSsFdsd 3hloirw der th sesm rore iho gom wfdowd bose ther  aith thy phe Nelrrlpzun ghte ste hgh  urlk
18950/18950 [=============

of a former child! O that record could wor the the the me the the sore the the the the the the the the the the the the the the the the the wher thar seare the the thar the the the the the me the the the the the the the the the the the the the seare the the the the selle sour the the the seare seere wine sor in the the the the the the the the be the the the the the the the the the the the the be the the the the the the tor ther sore sill
----- diversity: 0.5
----- Generating with seed: "of a former child! O that record could w"
of a former child! O that record could whas the sore bath ther palld, deor ther int onder the thou ghe than dound thou in be ther seree the math the woel, mare, That that on in lour sor arghe seoin sas th sore, Or thee, Whar sy ore wile the sare the thas ling thes the rive, Ar wand be mes the the ly ur thane thy the, the thy thet the the mon, beee fear dome ne, And thou be thaur is bou the fall loul thes soind woth sikes me wite thar th
----- diversity: 1.0
-----

e, A dearer birth than this his love hadp theifr, nuthe the tiall) wirtbus Iindy moir tis sise bee, for wineis tert mo cove bonve whyeall me owcong’s farkonr’s bat beis thouessise erewhey aur cay mile and in the love sot un the , ar thes dorddayt dor3 in my conght  ist ond lour my meal, diys mace poves onlead bad s the dterads you ced sist meon’s yolin eino thice tate Whive? . nris k lich thou drows thel winr ole ass. mor mine preay tho
----- diversity: 1.2
----- Generating with seed: "e, A dearer birth than this his love had"
e, A dearer birth than this his love hady golk mueo te, nod, ware not, arows, Thmce cakunsw sxill de rren tees Soust, theer taic, ) 6ckerot gsliki’s nack? I I wonen. I Satr ntyur-repvesss, doe snimy beveand wrase vyey as Lowce tracapstter’? Whate, Mu cfaifak w4taec in trtoufifis an tinn, Hove the, 3lont winse I7 bpreere’s Sist foli hat Pass sineed bsgoot s tat ah thim doplgpes weanis woel dacth warlemrs? O Ym! thu allvef or eakbey Thed 
18950/18950 [=============

lawful plea commence: Such civil war is that and the wore the that that the the my are hat the the beare in the me the rove the with the the will my are the leare the the the be my sore the love the the in the that thy meart in the the the with the the love the rover that the that the my are the mare the reart of the the the my arthe the will with the the sour the with the the in thou hour hat the my sores and the that an the me my mare
----- diversity: 0.5
----- Generating with seed: "lawful plea commence: Such civil war is "
lawful plea commence: Such civil war is ar hat thou hes in thou had so hest wor, and and, Ho the fare dear so the seans in thy with that whay in whit then with that that me love reare, Thes the mave the lave heake no the rong thar thy my aow gring the merss and and and so the lover that that fourd thy seinhes min the thar hange tho hared in wher what the in wot lfar to love, Than thou thas in thou the be ore porear the arle bes love the
----- diversity: 1.0
-----

uty was of yore. 69 Those parts of thee thas dey lowe. What houch sn wienco the, Whel avent my my bat oranst righare’s in my ordsend fot ongulg yous ton t Or dile, Thou min seive ptill deess wive mons deamen’t. d mwert and on whow tor thith Yis, of all be duer: Soed no orques bo fot lavt thy have whe I I hamit yyake my hase, nile the uumy nougheed beat doked my if my yes sall cel’vered, Thy bobenst youl no seer I than. Wroe ale iuch shy
----- diversity: 1.2
----- Generating with seed: "uty was of yore. 69 Those parts of thee "
uty was of yore. 69 Those parts of thee wull, Soke kwes his love fo mott wligh’t wfe, ’y wife a. Arothilidter haec, eeeen Thae sungers to bus me cadll I WhaFli lhabe de if qreangyd Wher gard, Af thou why aokkn Seey Uno benot not o hhimy, 6Lend hos !un  hen somy rint wresino fo Irs, Oy tar whow npes buserad is thons, Tol tcinged lover wsenv to ois eill’s pore, This swint pilh ireish stwalene o woich yous yad porite But wreches mfos’k bl 
18950/18950 [=============

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN